In [27]:
!pip install matplotlib_inline

In [28]:
import os

import joblib as jb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torchvision
from d2l import torch as d2l
from torch import nn
from torchmetrics import R2Score
from torchsummary import summary

In [29]:
class Flatten(nn.Module):
    """Converts N-dimensional tensor into 'flat' one."""

    def __init__(self, keep_batch_dim=True):
        super().__init__()
        self.keep_batch_dim = keep_batch_dim

    def forward(self, x):
        if self.keep_batch_dim:
            return x.view(x.size(0), -1)
        return x.view(-1)


In [59]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers)
        self.linear = nn.Linear(hidden_size, output_size)
        self.relu1 = nn.ReLU()
    
    def forward(self, x):
        # Set the initial hidden and cell states to be all zeros
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
    
        # Pass the input through the LSTM layer
        out, (h_n, c_n) = self.lstm(x, (h_0, c_0))
    
        # Select the last output of the LSTM layer as the final output
        out = out[:, -1, :]
    
        # Pass the final output through a fully-connected layer to get the final prediction
        out = self.linear(out)
    
        return out

In [31]:
# Calculating the loss
# loss = nn.CrossEntropyLoss(reduction='none')
loss = nn.MSELoss(reduction='none')

def evaluate_loss(data_iter, net, devices):
    l_sum, n = 0.0, 0
    for features, labels in data_iter:
        features, labels = features.to(devices[0]), labels.to(devices[0])
        outputs = net(features)
        l = loss(outputs, labels)
        l_sum += l.sum()
        n += labels.numel()
    return l_sum / n


In [32]:
def loss_plotter(train_losses,test_losses):
    plt.figure(figsize = (10,6) )
    plt.plot(train_losses, 'r*-', label = "Train Loss")
    plt.plot(test_losses, 'b*-', label = "Test Loss")
    plt.legend()
    plt.grid()
    plt.xlabel("Epochs")
    plt.ylabel("The Loss Value")
    

In [33]:
spo2_vals = jb.load('strided_dataset_spo2_vals.pk')

In [34]:
stride_dataset = jb.load('strided_dataset.pk')

In [35]:
dataset = pd.DataFrame.from_dict(stride_dataset).transpose()

In [36]:
dataset_spo2 = pd.DataFrame.from_dict(spo2_vals).transpose()

In [37]:
dataset_spo2 = dataset_spo2/100

In [38]:
X_train, X_valid, X_test = np.split(dataset.sample(
    frac=1), [int(.8*len(dataset)), int(.9*len(dataset))])
y_train, y_valid, y_test = dataset_spo2.loc[X_train.index], dataset_spo2.loc[
    X_valid.index], dataset_spo2.loc[X_test.index]


In [39]:
# unroll the data
X_train_unrolled = []
y_train_unrolled = []
for r in X_train.index:
    for c in X_train.columns:
        # skip values containing NaN
        if not np.isnan(X_train.loc[r][c]).any() and not np.isnan(y_train.loc[r][c]):
            X_train_unrolled.append(X_train.loc[r][c])
            y_train_unrolled.append(y_train.loc[r][c])

X_valid_unrolled = []
y_valid_unrolled = []
for r in X_valid.index:
    for c in X_valid.columns:
        # skip values containing NaN
        if not np.isnan(X_valid.loc[r][c]).any() and not np.isnan(y_valid.loc[r][c]):
            X_valid_unrolled.append(X_valid.loc[r][c])
            y_valid_unrolled.append(y_valid.loc[r][c])

X_test_unrolled = []
y_test_unrolled = []
for r in X_test.index:
    for c in X_test.columns:
        # skip values containing NaN
        if not np.isnan(X_test.loc[r][c]).any() and not np.isnan(y_test.loc[r][c]):
            X_test_unrolled.append(X_test.loc[r][c])
            y_test_unrolled.append(y_test.loc[r][c])


In [40]:
# convert to tensors
X_train, y_train = torch.tensor(X_train_unrolled, dtype=torch.float32), torch.tensor(
    y_train_unrolled, dtype=torch.float32)
X_valid, y_valid = torch.tensor(X_valid_unrolled, dtype=torch.float32), torch.tensor(
    y_valid_unrolled, dtype=torch.float32)
X_test, y_test = torch.tensor(X_test_unrolled, dtype=torch.float32), torch.tensor(
    y_test_unrolled, dtype=torch.float32)

In [41]:
# reshape y tensors
y_train = y_train.reshape(-1, 1)
y_valid = y_valid.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [42]:
# normalize x data
X_train_mean = X_train.mean() 
X_train_std = X_train.std()
X_train = (X_train - X_train_mean) / (X_train_std + 1e-7)

X_valid_mean = X_valid.mean()
X_valid_std = X_valid.std()
X_valid = (X_valid - X_valid_mean) / (X_valid_std + 1e-7)

X_test_mean = X_test.mean()
X_test_std = X_test.std()
X_test = (X_test - X_test_mean) / (X_test_std + 1e-7)

In [43]:
# convert to torch iterators
batch_size = 64
train_iter = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(
    X_train, y_train), batch_size=batch_size, shuffle=True)
valid_iter = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(
    X_valid, y_valid), batch_size=batch_size, shuffle=True)
test_iter = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(
    X_test, y_test), batch_size=batch_size, shuffle=True)

In [44]:
num_epochs = 10
lr, wd = 0.001, 1e-4
lr_period, lr_decay = 2, 0.9

In [45]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [60]:
model = RNNModel(input_size=240, hidden_size=32, num_layers=2, output_size=5).to(device)
#Setting the loss function
cost = nn.L1Loss()

#Setting the optimizer with the model parameters and learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

#this is defined to print how many steps are remaining when training
total_step = len(train_iter)

In [61]:
summary(model,(1,240))

RuntimeError: Expected hidden[0] size (2, 1, 32), got [2, 2, 32]

In [62]:
train_losses = []
test_losses = []

num_epochs = 250
for epoch in range(num_epochs):
    

    
    
    for i, (features, labels) in enumerate(train_iter):
        batch_size = features.shape[0]
        features = torch.reshape(features, (batch_size,1,240))
        labels = torch.reshape(labels, (batch_size,1,1))
        features = features.to(device)
        labels = labels.to(device)
        outputs = model(features)
        loss = cost(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        
        
    for i, (features_test, labels_test) in enumerate(test_iter):
        batch_size = features_test.shape[0]
        features_test = torch.reshape(features_test, (batch_size,1,240))
        labels_test = torch.reshape(labels_test, (batch_size,1,1))
        
        features_test = features_test.to(device)
        labels_test = labels_test.to(device)
        outputs_test = model(features_test)
        loss_test = cost(outputs_test,labels_test)
     
    train_losses.append(loss.item()*100)
    test_losses.append(loss_test.item()*100)
    print("1) The Train Loss value with the epoch: " + str(epoch) + " is equal to " + str(loss.item()*100))
    print("2) The Test Loss value with the epoch: " + str(epoch) + " is equal to " + str(loss_test.item()*100))
    print("-------------------------------------------------------------------")
    
    if epoch == num_epochs-2:
        break



RuntimeError: Expected hidden[0] size (2, 1, 32), got [2, 64, 32]

In [ ]:
loss_plotter(train_losses,test_losses)